# Terminal Module Tests

This notebook tests the functionality of the `terminal.py` module. We'll cover:

1. Creating a terminal session
2. Executing commands
3. Reading output
4. Deleting a session
5. Error handling

In [ ]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src.auto_coder_web.terminal import TerminalSession, TerminalManager

## 1. Testing Terminal Session Creation

In [ ]:
# Create a terminal manager instance
manager = TerminalManager()

# Create a new session
session_id = manager.create_session()
print(f"Created session with ID: {session_id}")

# Get the session
session = manager.get_session(session_id)
print(f"Session working directory: {session.working_directory}")

## 2. Testing Basic Commands

In [ ]:
# Test simple echo command
manager.execute_command(session_id, "echo 'Hello from terminal'")
output = manager.get_output(session_id)
print("Echo command output:", output)

## 3. Testing Directory Navigation

In [ ]:
# Test cd command
manager.execute_command(session_id, "cd /tmp")
session = manager.get_session(session_id)
print(f"New working directory: {session.working_directory}")

# Test pwd command
manager.execute_command(session_id, "pwd")
output = manager.get_output(session_id)
print("PWD output:", output)

## 4. Testing Command History

In [ ]:
# Execute a few commands
commands = ["ls", "date", "whoami"]
for cmd in commands:
    manager.execute_command(session_id, cmd)
    output = manager.get_output(session_id)
    print(f"Command '{cmd}' output:", output)

# Check command history
session = manager.get_session(session_id)
print("\nCommand history:")
for cmd in session.command_history:
    print(f"- {cmd}")

## 5. Testing Error Handling

In [ ]:
# Test invalid command
manager.execute_command(session_id, "invalidcommand")
output = manager.get_output(session_id)
print("Invalid command output:", output)

# Test cd to non-existent directory
manager.execute_command(session_id, "cd /nonexistent/directory")
output = manager.get_output(session_id)
print("Invalid cd output:", output)

## 6. Testing Session Cleanup

In [ ]:
# Delete the session
manager.delete_session(session_id)

# Try to get the deleted session
deleted_session = manager.get_session(session_id)
print("Deleted session:", deleted_session)  # Should be None

# Try to get output from deleted session
output = manager.get_output(session_id)
print("Output from deleted session:", output)  # Should be None

## 7. Testing Long-Running Commands

In [ ]:
# Create a new session for long-running command test
session_id = manager.create_session()

# Execute a command that produces output over time
manager.execute_command(session_id, "for i in {1..5}; do echo $i; sleep 1; done")

# Read output multiple times to demonstrate streaming
for _ in range(6):
    output = manager.get_output(session_id)
    if output:
        print("Received output:", output)
    import time
    time.sleep(1)

# Cleanup
manager.delete_session(session_id)